In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Data generation

We will first start by generating two clusters of data points. Then our task in this exercise is to define a boundary that separates the clusters, given a few observations from each cluster. When new data arrives, we can easily decide which cluster the new observation actually belongs to, i.e. **classifying** the new observation to one of the two clusters.   

*Run the below 3 code blocks and observe the two clusters of data points in the plot.*

In [2]:
cluster_centres = np.array([[-1, -3], [2, 2]])
colour_list = ['red', 'blue']

def generateData(num_samples_pc, dimensions=2, cluster_locs=cluster_centres):
    data = []
    labels = []

    for c, locs in enumerate(cluster_locs):
        pos = np.random.randn(num_samples_pc, dimensions) + locs
        data.append(pos)
        labels.append(np.ones(num_samples_pc) * c)

    data_np = np.concatenate(data, axis=0)
    labels_np = np.concatenate(labels, axis=0)
    return data_np, labels_np

train_data, train_labs = generateData(100)
test_data, test_labs = generateData(10)

In [3]:
def plot_data(data, labs, a=None, b=None, c=None, color=colour_list):
  plt.xlim([-8, 8])
  plt.ylim([-8, 8])
  for ic in range(2):
      ind_class = np.where(labs == ic)
      #color = plt.cm.Set1(ic)
      plt.scatter(data[ind_class, 0], data[ind_class, 1], s=10, color=color[ic], label=str(ic))

  if a is not None and b is not None and c is not None:
    x = np.linspace(-8, 8, 10)
    y = ( - b * x - c) / a
    plt.plot(x, y)

  plt.ylabel("x2")
  plt.xlabel("x1")
  plt.gca().set_aspect('equal')
  plt.show()

In [ ]:
plot_data(train_data, train_labs)

# Decision boundary

* The decision boundary is a line which separates out the data points from different clusters.

* If we have a data point $\mathbf{x}$ that we want to classify, where $\mathbf{x} = \{x_1, x_2\}$ -- a 2D feature vector. Then the predicted class depends on which side of the line $\mathbf{x}$ lies.

* We represent the decision boundary of the form: $f(\mathbf{x}) = ax_1 + bx_2 + c = 0$.

* This allows us to easily compute which side of the line the point lies from looking at the sign of $f(\mathbf{x}) = ax_1 + bx_2 + c$.

* A positive sign indicates that the data point $\mathbf{x}$ belongs to the blue class, and a negative sign indicates red.



## Sigmoid function

* The sigmoid function is defined as: $\sigma (t) =\frac{1}{1 + exp(-t)}$

* The sigmoid function has some very nice properties (see the lecture note). An important one is that the value of the sigmoid function always lies in between 0 and 1. Hence, it's a perfect tool to model probability distributions.

* Here, we use the sigmoid function to represent the probability that a data point $\mathbf{x}$ is from a certain class $C_i$, where $i$ is either 0 or 1.

## Task 1: Implement sigmoid function

* $\color{red}{\textbf{TO DO :}}$ Implement the sigmoid function in the code block below.*

Hint: Consider to use the `np.exp()` function.

In [ ]:
cluster_centres = np.array([[-1, -3], [2, 2]])

def sigmoid(x):
  ## TODO implement sigmoid in here ##
  ####
  ###
  return sigmoid_value

print(sigmoid(cluster_centres))

print(cluster_centres + 1)

Let's plot the sigmoid function using the function below to see its shape.

In [ ]:
t = np.linspace(-10, 10, 100)
sigmoid_t = sigmoid(t)
plt.plot(t, sigmoid_t, 'b-')
plt.xlim([-10, 10])
#plt.gca().set_aspect('equal')
plt.xlabel("input t")
plt.ylabel("sigmoid value")
plt.grid(True)
plt.show()

Now can we combine the sigmoid function with the equation of our linear decision boundary to get a predictive probability for the class labels of a 2D data point?

* Think about the output of the function $ay + bx + c$ and how that could work with the sigmoid function.

* HINT: If a point lies on the line $ay + bx + c = 0$ what is the probability that it belongs to each class?

## Task 2: Predictive probability of a data point

* $\color{red}{\textbf{TO DO :}}$ Implement a function which gives the predictive probability $p(\mathbf{x})$ of 2D data points $\mathbf{x}=(x_1, x_2)$ being in the blue class for the given values for `a`, `b` and `c`. You will need to use the sigmoid function and evaluate $ay + bx + c$.


In [ ]:
def predictive_prob(x, a, b, c):
  ## TODO ##
  ####
  ####
  ## TODO end ##
  return pred_prob

print(cluster_centres)
print(cluster_centres[:,1])

# Learning Objective: Binary Cross-Entropy Loss

* We want to find a way to automatically infer the decision boundary.

* The best decision boundary is the one which will assign a low probability of a data point being `blue` if it actually belongs to the `red` class and a probability of a data point being `red` if it does belong to the `red` class.

* We use the Binary Cross Entropy loss to work out the decision boundary.

$$
L = \sum_{i=1}^{N}- y_i\log(p(\mathbf{x}_i)) - (1 - y_i)\log(1 - p(\mathbf{x}_i))
$$

* Where $y_i$ represent $i$th element of `train_labs`, which is a vector of 0s and 1s. Here 0 indicates that the corresponding $i$th data point in `train_data` is `red` and 1 indicates that it is `blue`. $\mathbf{x}_i$ is a data point in `train_data` and $p(\mathbf{x}_i)$ represents our predictive probability that $\mathbf{x}_i$ is blue.


## Task 3: Implement the binary cross entropy loss.

* $\color{red}{\textbf{TO DO :}}$ Implement the BCE loss in the function below.*

In [ ]:
def bce_loss(pred_probs, y):
  ## TODO ##
  ###
  ###
  ## TODO end ##
  return bce

### Is BCE a good metric for how well our decision boundary seperates our data?

Play around with the vlaues of `a`, `b`, and `c` below to see how they effect the values of BCE loss.



In [ ]:
a=5
b=5
c=1

plot_data(train_data, train_labs, a, b, c)
print("BCE loss: " + str(bce_loss(predictive_prob(train_data, a, b, c), train_labs)))

# Optimization with Gradient Descent (GD)

* Gradient descent is a way to minimize (or maximize) a function, just think of it as walking down a hill.

* If you want to walk down a hill from a random point, you'd choose a direction which points down, and then take a step.

* That's what we do a lot in machine learning (literally, this is what everyone uses all the time), but rather than take a physical step, we just move the parameters by a certain amount which we call a step.

* For GD to be useful, we need to calculate the gradient of our loss function.

* The gradients for BCE are:

$$
\frac{\partial{L}}{\partial a} = \sum_{i=1}^{N}(\sigma(ax_1^i + bx_2^i + c) - y^i)x_1^i
$$

$$
\frac{\partial{L}}{\partial b} = \sum_{i=1}^{N}(\sigma(ax_1^i + bx_2^i + c) - y^i)x_2^i
$$

$$
\frac{\partial{L}}{\partial c} = \sum_{i=1}^{N}(\sigma(ax_1^i + bx_2^i + c) - y^i)
$$

Note that $(\mathbf{x}^i, y^i)$ refer to the $i$-th data points in our observations.


## Task 4: Implement the gradient of $a, b$ and $c$

* $\color{red}{\textbf{TO DO :}}$ Complete the below code block to implement a function which returns the gradients of $a, b$ and $c$.

You can return multiple values from a function using `return grad_a, grad_b, grad_c`

In [ ]:
def compute_gradients(x, a, b, c, y):
  ## TODO ##
  ###
  ###
  ###
  ## TODO end ##
  return grad_a, grad_b, grad_c

## Task 5: Implement the gradient descent algorithm.

* $\color{red}{\textbf{TO DO :}}$ You will need the following equation to take a step:

$$
a^{t+1} = a^{t} - \alpha\cdot\frac{\partial{L}}{\partial a^{t}}
$$

In [ ]:
def optimize(train_data, train_labs, num_steps):
    a, b, c = 2, -1, -5
    lr = 5e-3
    for i in range(num_steps):
      ## TODO ##
      ###
      ###
      ###
      ## TODO end ##

      if i % 10 == 0:
        print("Iteration %i\tLoss %.3f" % (i, bce_loss(predictive_prob(train_data, a, b, c), train_labs)))

    plot_data(train_data, train_labs, a, b, c)

    return a, b, c


a_opt, b_opt, c_opt = optimize(train_data, train_labs, 100)
print("The optimised parameters are: a=" + str(a_opt) + ", b=" + str(b_opt) + ", c=" + str(c_opt))

### What do you notice about the loss value? Is it doing what you expect?

# Evaluating a Classifier

We have now found out the parameters of a linear classifier. How well does it perform on the classification task on the 2D points? On what metric should we evaluate it on? Should we evaluate it on the **training set**, a set of data points that we use to estimate the classifier's parameters? Or should it be evaluated on a held-out **test set**, a set of data points that are reserved from training and only used for evaluating the model performance? What is the importance of having these two separate sets?

We explore these questions in this section.



*   We often use the accuracy of a classifier as the performance metric.
*   It is defined as: $ \text{Accuracy} = \frac{\text{Number of correct prediction}}{\text{Number of total prediction}}$



## Task 6: Test set

To evaluate a classifier, we first need to use it to make some predictions. At the beginning of this notebook, we generated some test data alongside of the training data. Check in the Data Generation section and look for two variables called `test_data` and `test_labs`.

In [ ]:
print(test_data.shape, test_labs.shape)
print("Test data:", test_data)
print("Test labels:", test_labs)

### Make prediction

* $\color{red}{\textbf{TO DO :}}$ can you make predictions using the $a, b, c$ values that you've found in Task 4b? Please complete the below code block.*  

In [ ]:
def predict_labels(x, a, b, c):
  ## TODO ##
  ###
  ###
  ## TODO end ##
  return pred_labels


test_labs_predicted = predict_labels(test_data, a=a_opt, b=b_opt, c=c_opt)
print("Predictions: " , test_labs_predicted)
print("Real labels: ", test_labs)

### Compute accuracy

With the predicted labels `test_labs_predicted` from the previous task and the ground truth test labels `test_labs`, can you now evaluate the classifier's accuracy?

* $\color{red}{\textbf{TO DO :}}$ complete the below code block that computes the classifier's accuracy.

In [ ]:
## TODO ##
accuracy = ###
## TODO end ##
print(accuracy)